# 1

#### En el ejemplo de teoría hemos visto la forma de entrenar utilizando un trozo de datos como en la imagen "way1.png". 

(Básicamente, tener en cuenta más datos de entrenamiento [[20% de test]])

Investiga la forma de realizar el ejercicio a partir del gif "way3_sliding_window.gif":

- ¿Da mejores resultados entrenar el modelo así?

Algo muy típico en el trabajo de data scientist

![](../rnn/sliding_windows_google.png)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import Callback

In [4]:
import tensorflow

In [5]:
tensorflow.__version__

'2.5.0'

In [2]:
# Total time points
N = 3000
# Time point to partition train/test splits
Tp = 750   

t=np.arange(0,N)
x=(2*np.sin(0.02*t)*np.sin(0.003*t))+0.5*np.random.normal(size=N)
df = pd.DataFrame(x, columns=['Data'])
df.head()

,Data
0,-0.144808
1,1.169739
2,-0.282285
3,0.415015
4,-0.818219


In [92]:
window_length = 500

def moving_window(df, window_length):
    # Retorna una lista de arrays de longitud variable.
    window_list = []
    for i in range(0, len(df)+1-window_length, window_length):
        window_list.append(np.array(df.values[:i+window_length]))
    return window_list

def window_split(data, train_size=0.8):
    # Divide cada array de la lista en una lista de tuplas con array de train y array de test.
    train_test_tuple_list = []

    for array in data:
        Tp = round(len(array)*train_size)
        train = array[:Tp, :]
        test = array[Tp:len(array), :]
        train_test_tuple_list.append((train, test))

    return train_test_tuple_list

def add_step(train_test_tuple_list, step=4):
    # Añade los elementos extra del step a cada conjunto de arrays de la lista
    added_step = []

    for train, test in train_test_tuple_list:
        test = np.append(test,np.repeat(test[-1,],step))
        train = np.append(train,np.repeat(train[-1,],step))
        added_step.append((train, test))

    return added_step

def convertToMatrix(data, step):
    X, Y =[], []
    for i in range(len(data)-step):
        d=i+step  
        X.append(data[i:d,])
        Y.append(data[d,])
    return np.array(X), np.array(Y)

def convertToWindowMatrix(train_test_tuple_list, step=4):
    # Retorna 4 listas de arrays.
    trainX, trainY, testX, testY = [], [], [], []
    for i in range(len(data)):
        tra_x, tra_y = convertToMatrix(train_test_tuple_list[i][0], step)
        trainX.append(tra_x)
        trainY.append(tra_y)
        tes_x, tes_y = convertToMatrix(train_test_tuple_list[i][1], step)
        testX.append(tes_x)
        testY.append(tes_y)

    return trainX, trainY, testX, testY

def add_dimension(arrays):
    for i in range(len(arrays)):
        arrays[i] = np.reshape(arrays[i], (arrays[i].shape[0], 1, arrays[i].shape[1]))
    return arrays

In [93]:
def preprocess_data(df, window_length=500, train_size=0.8, step=4):

    # lista de arrays de longitud variable. Se crean las ventanas
    data = moving_window(df, window_length)

    # Divide cada array de la lista en una lista de tuplas con array de train y array de test.
    train_test_tuple_list = window_split(data)

    # Añade los elementos extra del step a cada conjunto de arrays de la lista.
    train_test_tuple_list = add_step(train_test_tuple_list, step=step)
    
    # Se crean los 4 conjuntos de arrays.
    trainX, trainY, testX, testY = convertToWindowMatrix(train_test_tuple_list, step=step)

    # Se redimensionan los arrays X.
    trainX = add_dimension(trainX)
    testX = add_dimension(testX)

    return trainX, trainY, testX, testY

In [94]:
trainX, trainY, testX, testY = preprocess_data(df)

In [95]:
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM

def build_simple_rnn(num_units=128, embedding=4, num_dense=32,lr=0.001):
    """
    Builds and compiles a simple RNN model
    Arguments:
              num_units: Number of units of a the simple RNN layer
              embedding: Embedding length
              num_dense: Number of neurons in the dense layer followed by the RNN layer
              lr: Learning rate (uses RMSprop optimizer)
    Returns:
              A compiled Keras model.
    """
    model = Sequential()
    # Long short term memory
    model.add(LSTM(units=num_units, input_shape=(1,embedding), activation="relu"))
    model.add(Dense(num_dense, activation="relu"))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer=RMSprop(learning_rate=lr),metrics=['mse'])
    
    return model

In [96]:
model = build_simple_rnn()

In [97]:
class MyCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        if (epoch+1) % 50 == 0 and epoch>0:
            print("Epoch number {} done".format(epoch+1))

In [99]:
batch_size = 16
num_epochs = 1000

In [101]:
dic = {'Window':[], 'MSE_Score':[]}

for i in range(len(trainX)):
    print('Fitting for Window:', i)
    model.fit(trainX[i],trainY[i], 
            epochs=num_epochs, 
            batch_size=batch_size, 
            callbacks=[MyCallback()],verbose=0)
    loss, mae = model.evaluate(testX[i], testY[i])
    print('Score:', score)
    dic['Window'].append(i)
    dic['MSE_Score'].append(mae)

results = pd.DataFrame(dic, index=range(len(dic['Window'])))

Fitting for Window: 0
Epoch number 50 done
Epoch number 100 done
Epoch number 150 done
Epoch number 200 done
Epoch number 250 done
Epoch number 300 done
Epoch number 350 done
Epoch number 400 done
Epoch number 450 done
Epoch number 500 done
Epoch number 550 done
Epoch number 600 done
Epoch number 650 done
Epoch number 700 done
Epoch number 750 done
Epoch number 800 done
Epoch number 850 done
Epoch number 900 done
Epoch number 950 done
Epoch number 1000 done
4/4 [==============================] - 0s 1ms/step - loss: 0.5806 - mse: 0.5806
Score: [0.5805737376213074, 0.5805737376213074]
Fitting for Window: 1
Epoch number 50 done
Epoch number 100 done
Epoch number 150 done
Epoch number 200 done
Epoch number 250 done
Epoch number 300 done
Epoch number 350 done
Epoch number 400 done
Epoch number 450 done
Epoch number 500 done
Epoch number 550 done
Epoch number 600 done
Epoch number 650 done
Epoch number 700 done
Epoch number 750 done
Epoch number 800 done
Epoch number 850 done
Epoch number 90

In [104]:
import matplotlib.pyplot as plt

plt.figure(figsize=(7,5))
plt.title("RMSE loss over epochs",fontsize=16)
plt.plot(np.sqrt(model.history.history['loss']),c='k',lw=2)
plt.grid(True)
plt.xlabel("Epochs",fontsize=14)
plt.ylabel("Root-mean-squared error",fontsize=14)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.show()

KeyError: 'loss'

In [107]:
results

,Window,Score
0,0,0.580574
1,1,0.916016
2,2,0.660724
3,3,0.933398
4,4,0.795169
5,5,0.756119
